In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import zipfile
import os

dataset_zip_path = "/content/drive/My Drive/Toptal/statues-dataset-processed.zip"
output_base = "/content/statues-dataset-processed"

# Unzipping the dataset
with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_base)

In [4]:
!pip install ultralytics==8.0.186 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.4/618.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.4 MB/s eta 0:00:00


In [5]:
import yaml

# Define the configuration dictionary
config = {
    'path': output_base,
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'names': {
        0: 'other',
        1: 'lenin',
        2: 'ataturk',
    }
}

# Define the output file path
yaml_path = 'statues_yolo8.yaml'

# Write the configuration dictionary to a YAML file
with open(yaml_path, 'w') as f:
    yaml.dump(config, f)

print(f"YAML configuration file '{yaml_path}' has been created.")

YAML configuration file 'statues_yolo8.yaml' has been created.


In [6]:
from ultralytics import YOLO
from wandb.integration.ultralytics import add_wandb_callback
import wandb

# Initialize a Weights & Biases run
wandb_project_name = "statues"
wandb.init(project=wandb_project_name, job_type="training")

# Define the YOLOv8 model
model_name = "yolov8s"
model = YOLO(f"{model_name}.pt")

# Add W&B callback for ultralytics
add_wandb_callback(
    model,
    enable_model_checkpointing=True,
    enable_train_validation_logging=False,
    enable_validation_logging=False,
)

# Train the model
model.train(
    project=wandb_project_name,
    data=yaml_path,
    epochs=50,
    batch=32,
    imgsz=1280,
    seed=22,
    deterministic=True,
)

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


100%|██████████| 6.23M/6.23M [00:00<00:00, 319MB/s]
New https://pypi.org/project/ultralytics/8.1.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.186 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=statues_yolo8.yaml, epochs=50, patience=50, batch=32, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=statues, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=22, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffe

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f568194ad40>
fitness: 0.7464833564798662
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.70552,     0.81917,     0.67863])
names: {0: 'other', 1: 'lenin', 2: 'ataturk'}
plot: True
results_dict: {'metrics/precision(B)': 0.8069878019066037, 'metrics/recall(B)': 0.7845842475472105, 'metrics/mAP50(B)': 0.8548916133103776, 'metrics/mAP50-95(B)': 0.7344379946098093, 'fitness': 0.7464833564798662}
save_dir: PosixPath('statues/train')
speed: {'preprocess': 0.48238191849146134, 'inference': 1.724573282095102, 'loss': 0.0004646105644030449, 'postprocess': 0.9142374381040915}

In [7]:
model.val()

Ultralytics YOLOv8.0.186 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients
val: Scanning /content/statues-dataset-processed/labels/val.cache... 40 images, 0 backgrounds, 1 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/statues-dataset-processed/images/val/Kemer_0749.jpg: ignoring corrupt image/label: negative label values [ -0.0032051]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.40s/it]
                   all         39         54      0.807      0.784       0.84      0.737
                 other         39         33      0.898      0.797      0.924      0.711
                 lenin         39         18      0.883      0.889      0.923      0.826
               ataturk         39          3       0.64      0.667      0.674      0.674
Speed: 0.5ms preprocess, 23.1ms inference, 0.0ms l

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f56444a0fd0>
fitness: 0.7473221367314307
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.71072,      0.8259,     0.67435])
names: {0: 'other', 1: 'lenin', 2: 'ataturk'}
plot: True
results_dict: {'metrics/precision(B)': 0.8069050757714241, 'metrics/recall(B)': 0.7840388777229624, 'metrics/mAP50(B)': 0.8403247696769166, 'metrics/mAP50-95(B)': 0.7369885108485988, 'fitness': 0.7473221367314307}
save_dir: PosixPath('statues/val')
speed: {'preprocess': 0.48830570318760014, 'inference': 23.11228483151167, 'loss': 0.0007702754094050481, 'postprocess': 0.7573641263521635}